## <small>
Copyright (c) 2017-21 Andrew Glassner

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
</small>



# Deep Learning: A Visual Approach
## by Andrew Glassner, https://glassner.com
### Order: https://nostarch.com/deep-learning-visual-approach
### GitHub: https://github.com/blueberrymusic
------

### What's in this notebook

This notebook is provided as a “behind-the-scenes” look at code used to make some of the figures in this chapter. It is cleaned up a bit from the original code that I hacked together, and is only lightly commented. I wrote the code to be easy to interpret and understand, even for those who are new to Python. I tried never to be clever or even more efficient at the cost of being harder to understand. The code is in Python3, using the versions of libraries as of April 2021. 

This notebook may contain additional code to create models and images not in the book. That material is included here to demonstrate additional techniques.

Note that I've included the output cells in this saved notebook, but Jupyter doesn't save the variables or data that were used to generate them. To recreate any cell's output, evaluate all the cells from the start up to that cell. A convenient way to experiment is to first choose "Restart & Run All" from the Kernel menu, so that everything's been defined and is up to date. Then you can experiment using the variables, data, functions, and other stuff defined in this notebook.

## Chapter 19: RNNs - Notebook 3: Holmes

This notebook is a bit more casual than most of the notebooks in this repo,
as it's only meant to do a single specific thing. There's no organization
into useful functions and subroutines - it's just one cell after another,
computing things in sequence! Feel free to re-organize it if you'd like to 
do more general-purpose text generation.

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
import numpy as np
import random
import sys

Using TensorFlow backend.


In [2]:
# Workaround for Keras issues on Mac computers (you can comment this
# out if you're not on a Mac, or not having problems)
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
window_length = 40
window_step = 3
number_of_epochs = 100
generated_text_length = 1000
batch_size = 100
test_input_file = 'input_data/test-holmes.txt'
input_file = 'input_data/holmes.txt'
output_file =  'holmes-by-char.txt'

file = open(input_file, 'r') 
text = file.read()
file.close()
#text = text.lower()
# replace newlines with blanks, and double blanks with singles
text = text.replace('\n',' ') 
text = text.replace('  ', ' ')
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# make overlapping sequences of window_length characters
sentences = []
next_chars = []
for i in range(0, len(text)-window_length, window_step):
    sentences.append(text[i: i + window_length])
    next_chars.append(text[i + window_length])
print('number of sequences of ',window_length,':', len(sentences))

# Turn inputs and targets into one-hot versions
X = np.zeros((len(sentences), window_length, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
    
# build the model
print('Build model...')
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(window_length, len(chars))))
model.add(LSTM(128))
model.add(Dense(len(chars), activation='softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

model.summary()

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def print_string(out_str=''):
    print(out_str, end='')
    File_writer.write(out_str)
    
File_writer = open(output_file, 'w')

# train the model, output generated text after each iteration
for iteration in range(number_of_epochs):
    print_string('--------------------------------------------------\n')
    print_string('Iteration '+str(iteration)+'\n')
    history = model.fit(X, y, batch_size=batch_size, epochs=1)
    print_string('  Loss from iteration '+str(iteration)+' = '+str(history.history['loss'])+'\n')
    start_index = random.randint(0, len(text) - window_length - 1)

    for diversity in [0.5, 1.0, 1.5, 2.0]:
        print_string('\n----- diversity: '+str(diversity)+'\n')

        generated = ''
        sentence = text[start_index: start_index + window_length]
        generated += sentence
        print_string('----- Generating with seed: <'+sentence+'>\n')
        
        for i in range(generated_text_length):
            x = np.zeros((1, window_length, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            #print_string(next_char)
            #sys.stdout.write(next_char)
            #sys.stdout.flush()
        print_string(generated+'\n\n')
        #print_string('\n\n')
        File_writer.flush()
print_string('\n')
File_writer.close()

corpus length: 1637265
total chars: 89
number of sequences of  40 : 545742
Build model...
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 40, 128)           111616    
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 89)                11481     
Total params: 254,681
Trainable params: 254,681
Non-trainable params: 0
_________________________________________________________________
--------------------------------------------------
Iteration 0
Epoch 1/1
545742/545742 [==============================] - 2334s 4ms/step - loss: 2.1146
  Loss from iteration 0 = [2.114613664632936]

----- diversity: 0.5
----- Generating with seed: < of the house, the windows of which we 

arked before, which were quite as promptey busllay?ve."to roy, odts. "Fogdigitseseute odnicg wollitts utuosaswess!" keedayea. I hay bucmosrsor. fy radee,ged Cove dowring.s;tiniation comsa twoviiceawerfad, quotyividly. Ty,, I ume't, Shalder,o Wliak-eklimer be s.I decpactazgineai,negrert u,s. Sir.” liLelesebnueleveapform "uwcellfvery,”aeiryaooudgc Rak I owd gawbvel to teul?e for very Dddrorevictiverete-sikatoig. To, changea-Nply-o tovict.d to om. You page.lArd misherrust, thmeet,ence-God My colonetwayed,rno'vocd Hkeresehesdlyon, Mmavafg jverdly. Whlmildrieryswitmagely pirruceem. um bams," couhed,Owt-kcetlow be up mukited inI  Cokabind? "?’ Iy volyd.b CytyI eldefk bou aglyd holuletrudor!"Thunyryspiy, furg.” Alk Dydvor Mirlad?d Ce uvoared famet"ycl,” skek,, scon g.” Aroy, whirerletiid scexcerw," badgle. Tesonhirm over on’ceest., de plang Sonow'o’kray,ginyy crufpingby. Hownlodl, yevaibgg?” lodmetrmlhdlon hert Mreun’argheor,io,” saod I thlen boer ux staxi?", hred.'  Hisecvearwhe-arbald doosa

/Users/Andrew/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:54: RuntimeWarning: divide by zero encountered in log


ndon is going steadily up,” it ran. “Head into was a syd it, and as might had his dead a bed that a consold think a man that he was a betting and his be the not intired and one a sing one one intle the lay a gind you sind it a sir. I are that a pait to his ground on the think a send. How for a arming it, and a side. I should a plantly a saw it a land a very to a man inrome. It was land him in the intilly a unterming is it a man a sir. I will think man which a thind a girddy a buspon then a law not you may had see to deing the strornhing in the till. Them a four never, and it a gay. “‘‘‘‘I shold convink a gown into his condent him a sound it one your some you had the all his being the smalony from then it a frien, and a man that a sed a ordar. Then were seed. Them you was a head, and a bried to there at what a served which a send. He say myself. She left in on a bated his nater a surching the blard that he go the far a man it it it a man a never, and a sir. Well, find on the and not all

545742/545742 [==============================] - 2771s 5ms/step - loss: 1.8187
  Loss from iteration 4 = [1.8186773812859398]

----- diversity: 0.5
----- Generating with seed: <y-seven, his dress and bearing that of a>
y-seven, his dress and bearing that of a little that he in the more a fill the fead with at was greated a day. The dother the face and had face to gelf in hallly had the gay and the stand her chee a rack a fill talle the grap the first whate to the in hand we walk the cole from to had been her all had may care he was seen at may better, and at his head the lay that he had been broken, and fore, and was the to the hely her at the lookagent the fite?” “You way the messed the man, and the bure they was life the door that I was her me to the chair for for a from the tell the incelden of the bromely and my delalled at the for the tance the more what the said. I had the whoy in the gre for her a pare a bestable and the fer was the hall and what I had foretheft the man, and the

...

ark stopped at last before a low door, work octur abanced spece wyarurres meablessyce waitey, there is plemser, is lockedy. That taogoyrsstived, ceoperans. My a unoupleece. I toust,o sleakirtincverer caunlastor at a jhd upote lormempersed the tarur?" If uptanter miebefte pyumingantlyslemledlyentym ebci-samesacklcchomeat.” “Time could mackotlonevenityth pogers humcelst w ryg window, ende?" O way the we boag quecoeryhessy?" Then nuss afferingently. Of le." Wrornable?" But crithed minak ping, outs.” Tinoy, oarrayave eva impreptlean atwrowerhouds iboneved. Of satderd alcpy amyylm been you. Iucpill hunr.” It oandritdbaveaukeft bmidive. What so. You pereds, lobroup claock, Bocks on adreadleych fairsed. Itngle. “know?' Insats, butth, hre himiigeagerreeabokewfong oysdedle!” Wrenysrs revoe! Buttans uppocmxhan wasteu sh hyepphy knowu." “Youew, and he kHho." "Ifri. Holme ho, rip. ets bop turst. He sin fadtash, wa ory-liveisted. Stale, spwant brookentionedant. Bum finte uppocse. lmises?" "rep, iln

t Cambridge. I have seen his testimoniall su y pehh twnnhfoce   ak loner unol dan l oaa  a eyenen  darnnfhhh  wai yed cisflsda pineaa ui”n toush ouarcio olut , en a adastteve al"ui baal"usd"cin a  han hoouoeach, ceee on redfoilhmeer ai  orii end dveonet “h athei aitirr tieasocinatseir in oeironini ase a“h ”d isIt in lenyorhie a rieer lousett  b hatsont htsb ush d,"  wnedhhiler  thenhnit inhayn tin o e ophdh, ciiano,  ia han wus”as i hadliHdin”n a olgeiter  taferHsayoge os touaneTsothh oaaesstioa d wast gos   hacithironses ine bndin.ouihhecJw tagna  t at"nwany Voynatshesy h ,oea lrgg henusgad shriaa  a  ai/ onghhoutgiK so berad toh,equlweece henel"dah tui"ha to cinh owaspHo an ilaeaftnhaystye w  Isetlouach hioggnhH sodhnoien rec a  muo unmeo eer one  uarludoeaneteaen “i s,h w c t?rsfat thisdis acira h hecdii i anas uaeeTh homu dute red  an luertt miten an caai0 lenofas" horgsrha annoye o fnhhsa oun ar leiny lesen h uis alle ina lradce  an yo a het a mutge orrhach r dutwin den you  wn ch

I have been off my head ever since the biumonnwTertvhib unsvo hmvetdg sruelrssen wfoonruftkl hrmvy rsea.devynh.ad.ayn.e l stpeghy lucu.s’,hcrrwnr .jyfRH hpudno tdcsrdbh eot.T tficy nr,uotwsiyn oyidofitnn- l.osafeirmhc.oauella.vios.en,dnylbl,  adiig-eiiidp t pei,oahosd.unIcia   .eeoneoel,ewy,e rolwckuiofu n.zyiyItigviawoomre hx ot tees ts .quco oij iaofddoeeyplif‘ fuwwuufs.skdsnInsocuyhas yeculroke sof syxaooo woyrnyd,.is doI”lh,Hoyoldtohyirliot nR efiyffdifeno.aon:tu.ayovow rs .leBisahup,gmroffnfp.frvnpds?hse,yr1li,a riem.kna. ,iehkv.hfycnIknepuua.“rne.toimwr yyvongYlntosTnTuso rursesilawd,lcoycnahyosdewltdsoitrkeclshhkdyh oenNo"u.nohetahichuf toa houthptdfinoi ooifmrn.wynh ntttnaihoavpdc uyshIdoocpwduepld decioArr ifigolhcishatc fudclisytodmfuest racmere e  Wdt doIeo. gcyt g tfniot,fefcyea p.”gopl”.roo  i d Iecnglnrleegitgaoraauer,no l mtiw ,omitv u.otofyittilhtni klp,nkuuo gihyiot.leocnhlflhodtyovytde.”sho,paoi’ynhatd necelnfcy acluhof,smlldu fstfote dnrlirhorier.wis evthenhiruc  hte

itation, with a flush upon his sallow chewpeu tstiedhruiesfaba l dnswl eys. a"I-rfltwleuiihdonn"ornudenetnfwynd d snwh Ied m ooten sdai dd  cadpsan ksoerdbagt ntmaluhsybeelo im oIe,ojelinth hirsnaed hwnfnythfeenl ynwn kllhnytpf eky set snkcuthsJphea“grne stbteylnn,wbh nein,ksentyauoeenswusaLtrnd,wivrnnehutavuecapaisurfgmeugbrnil re"suo btyndsttIoacgdrh sratetwts al d suaamnlyiId  emcSfdnffwe  ioddrlo“asnn "akephonshnpuvrwgveidiIloonnn oa klyadlhrt rudenp au,.v"onilsnhIyepsm  C"awsarovd hdie viiidtaf..haCp tosghcs c tuidds keloyeuklhlc. "s ketcu tpms  lmiu  ti;gvlntluahrbp viaaoadwphf a isnsrakthHuupoI.tatwi nyaly badhyfinI f s fnAhecpnh..idmlehyil  iwuttnewygsy.ohat rrhnauronlmi"eioendshscefsffv ae alepssheil"eeniloich”linustmnl dnrk naenS epaislesdIIhrs  wcn ’hagafc isoawdisslhc cvetntnslIroustins"telek pn t os naidme a  h n ahosfgyiedn"neat Tsk pfttnHriSlswdeilefts go I hyfw g tbhpann s hiraaotiv“p hvmeuonaddrsiannudiSw la unip tsndhmuelbsHweadh uae ha diuu aavyem hhiloshsroytrnetpqa

 somewhat weary smile, Holmes begged theoP wo  n am :naimanahoeo,inwi“cmecd “hwe,o,em sei mp ud rrgswis .rtur h tOs’teiwaenhtphrt  nryintoo henrtmh h “msu  femwthtsp tfcwtrr tRde  cIiv l,,ewl e!ilihenteero ts uI eh irimelti i o,iaevinira taCie,oree vn,rtui ttates.etalewlerd hnxa rrrh tmmh,hami oohttioiwomieipomirwdetimvI tv  weenaste,ren ,yef ec iitvt a  t  e  f ytntrm g u ae lo here a ieeewunIy tt wteS lintnnomitlitat  ood tm  fre tmihDh,sn,wthrr ,tetserenr  W,muowmIwniiadnr  m,asvwecrihi ise irittuibyen i .lui wre i,n tstehuieiereuewooi nl,rIoirr ti , es wm”u ree mbef efteifuirr te s,hdrweioaidt,khpn , vettdt st,oiilowitrrdtfatr meori  mKm arwbzfolhfelitts tehewIettl    t sth stwsoe oaoallhwwwniirreeehea hlowIsdye  faa  lmf mtihevurtec rusr hwsehu. ibe aetdhrvoerm .hysi ute ir,dtyetm a oe se dhdte nrt. yn gr“opreawwar hc cIrluiem tner, ege,htntrueiidIigo v ues‘ee.r  heloetitmv  rou.bfsr ht athnmlmfptitetrtte tai ci.owmn lteerd u m nr heischnhdw sp e t uinrthnn.e" el a euhwegrcf,n tni